# Lab 2: Machine Learning with PyTorch - SOLUTIONS

## From Classical ML to Deep Learning Foundations

**Duration:** 90-120 minutes | **Difficulty:** Intermediate | **Prerequisites:** Lab 1

**This notebook contains all solutions. Use for reference after attempting the exercises.**

---

## Overview

This lab bridges classical machine learning and deep learning by teaching PyTorch fundamentals through hands-on implementation of ML algorithms. You'll learn to build, train, and evaluate models using the same patterns used in production deep learning systems.

### Lab Structure

| Part | Topic | Key Concepts |
|------|-------|--------------|
| **Part 1** | PyTorch Tensors | Creating tensors, tensor operations, automatic differentiation (autograd) |
| **Part 2** | Linear Regression | Training loop from scratch, MSE loss, nn.Module, gradient descent |
| **Part 3** | Logistic Regression | Sigmoid function, BCE loss, binary classification, decision boundaries |
| **Part 4** | Support Vector Machines | Kernel trick (linear, RBF, polynomial), margins, support vectors |
| **Part 5** | Model Evaluation | Confusion matrix, precision, recall, F1-score, classification report |

### Key Pattern You'll Learn

The PyTorch training loop used in all deep learning:

```python
for epoch in range(n_epochs):
    y_pred = model(X)           # Forward pass
    loss = criterion(y_pred, y) # Compute loss
    optimizer.zero_grad()       # Clear gradients
    loss.backward()             # Backward pass
    optimizer.step()            # Update weights
```

---

In [ ]:
# SETUP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.datasets import make_classification, make_moons

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

np.random.seed(42)
torch.manual_seed(42)

print("Setup Complete!")

---
# Part 1: PyTorch Tensors - Solutions
---

In [ ]:
# ============================================
# EXERCISE 1.1 SOLUTION: Create Tensors
# ============================================

# a) Create a tensor containing [10, 20, 30, 40, 50]
tensor_a = torch.tensor([10, 20, 30, 40, 50])

# b) Create a 3x4 tensor filled with zeros
tensor_b = torch.zeros(3, 4)

# c) Create a 2x5 tensor with random values (uniform)
tensor_c = torch.rand(2, 5)

# ---- Test ----
print(f"a) {tensor_a}")
print(f"b) Shape: {tensor_b.shape}")
print(f"c) Random tensor with shape {tensor_c.shape}")

In [ ]:
# ============================================
# EXERCISE 1.2 SOLUTION: Tensor Operations
# ============================================

x = torch.tensor([[1., 2., 3.], [4., 5., 6.]])
y = torch.tensor([[7., 8., 9.], [10., 11., 12.]])

print("x =")
print(x)
print("\ny =")
print(y)
print()

# a) Add x and y element-wise
result_add = x + y

# b) Calculate the mean of x
result_mean = x.mean()

# c) Calculate the sum of each row of x (dim=1)
result_row_sum = x.sum(dim=1)

# d) Reshape x to be 3 rows x 2 columns
result_reshape = x.reshape(3, 2)

# ---- Test ----
print(f"a) x + y =\n{result_add}")
print(f"\nb) Mean of x = {result_mean}")
print(f"\nc) Sum of each row = {result_row_sum}")
print(f"\nd) x reshaped to 3x2:\n{result_reshape}")

In [ ]:
# ============================================
# EXERCISE 1.3 SOLUTION: Practice with Autograd
# ============================================

# Create x = 2.0 with gradient tracking
x = torch.tensor([2.0], requires_grad=True)

# Compute y = 3x² - 4x + 5
y = 3*x**2 - 4*x + 5

# Compute the gradient
y.backward()

# ---- Test ----
print(f"y = 3x² - 4x + 5 at x=2: y = {y.item()}")
print(f"dy/dx = 6x - 4 at x=2: dy/dx = {x.grad.item()}")

---
# Part 2: Linear Regression - Solutions
---

In [ ]:
# Setup: Generate data
np.random.seed(42)
n_samples = 100
X_np = np.random.rand(n_samples, 1) * 10
y_np = 3 * X_np + 2 + np.random.randn(n_samples, 1) * 1.5
X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32)

In [ ]:
# ============================================
# EXERCISE 2.1 SOLUTION: Build Linear Regression
# ============================================

torch.manual_seed(123)

# Define the model
class MyLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # SOLUTION: Create a linear layer (1 input, 1 output)
        self.linear = nn.Linear(1, 1)
    
    def forward(self, x):
        # SOLUTION: Return the output of the linear layer
        return self.linear(x)

# Create model, loss function, and optimizer
my_model = MyLinearRegression()
my_criterion = nn.MSELoss()  # SOLUTION
my_optimizer = optim.SGD(my_model.parameters(), lr=0.01)  # SOLUTION

# Training loop
my_losses = []
for epoch in range(100):
    # SOLUTION: Complete the training loop
    # 1. Forward pass
    y_pred = my_model(X)
    
    # 2. Compute loss
    loss = my_criterion(y_pred, y)
    
    # 3. Zero gradients
    my_optimizer.zero_grad()
    
    # 4. Backward pass
    loss.backward()
    
    # 5. Update weights
    my_optimizer.step()
    
    my_losses.append(loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch {epoch+1}/100 | Loss: {loss.item():.4f}')

# Print results
print(f"\nLearned: w = {my_model.linear.weight.item():.4f}, b = {my_model.linear.bias.item():.4f}")
print(f"True:    w = 3.0000, b = 2.0000")

---
# Part 3: Logistic Regression - Solutions
---

In [ ]:
# Setup: Generate classification data
X_class, y_class = make_classification(
    n_samples=300, n_features=2, n_redundant=0, n_informative=2,
    n_clusters_per_class=1, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
y_test_t = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

In [ ]:
# ============================================
# EXERCISE 3.1 SOLUTION: Build a Classifier
# ============================================

torch.manual_seed(42)

class MyClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        # SOLUTION: Create linear layer and sigmoid
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # SOLUTION: Apply linear then sigmoid
        z = self.linear(x)
        return self.sigmoid(z)

# Create model, loss, and optimizer
my_classifier = MyClassifier(input_dim=2)
my_bce_loss = nn.BCELoss()  # SOLUTION
my_opt = optim.Adam(my_classifier.parameters(), lr=0.1)  # SOLUTION

# Training loop
for epoch in range(100):
    # SOLUTION: Complete the training loop
    y_pred = my_classifier(X_train_t)
    loss = my_bce_loss(y_pred, y_train_t)
    
    my_opt.zero_grad()
    loss.backward()
    my_opt.step()
    
    if (epoch + 1) % 20 == 0:
        acc = ((y_pred >= 0.5).float() == y_train_t).float().mean()
        print(f'Epoch {epoch+1}/100 | Loss: {loss.item():.4f} | Acc: {acc.item():.4f}')

# Test accuracy
my_classifier.eval()
with torch.no_grad():
    y_pred = my_classifier(X_test_t)
    y_class = (y_pred >= 0.5).float()
    acc = (y_class == y_test_t).float().mean()
print(f"\nTest Accuracy: {acc.item():.4f}")

---
# Part 4: Support Vector Machines - Solutions
---

In [ ]:
# Setup: Generate moons data
X_moons, y_moons = make_moons(n_samples=300, noise=0.2, random_state=42)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_moons, y_moons, test_size=0.2, random_state=42
)

In [ ]:
# ============================================
# EXERCISE 4.1 SOLUTION: Train an SVM
# ============================================

# a) Create an RBF SVM with C=10
my_svm = SVC(kernel='rbf', C=10)  # SOLUTION

# b) Fit it on the training data
my_svm.fit(X_train_m, y_train_m)  # SOLUTION

# c) Calculate test accuracy
accuracy = my_svm.score(X_test_m, y_test_m)
n_support = len(my_svm.support_vectors_)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Number of Support Vectors: {n_support}")

---
# Part 5: Model Evaluation - Solutions
---

In [ ]:
# Setup: Train a logistic regression model for evaluation
torch.manual_seed(42)

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

log_model = LogisticRegressionModel(input_dim=2)
criterion_bce = nn.BCELoss()
optimizer_log = optim.Adam(log_model.parameters(), lr=0.1)

for epoch in range(100):
    y_pred_prob = log_model(X_train_t)
    loss = criterion_bce(y_pred_prob, y_train_t)
    optimizer_log.zero_grad()
    loss.backward()
    optimizer_log.step()

In [ ]:
# ============================================
# EXERCISE 5.1 SOLUTION: Evaluate Your Model
# ============================================

# Get predictions from the logistic regression model
log_model.eval()
with torch.no_grad():
    y_pred_prob = log_model(X_test_t)
    y_pred_log = (y_pred_prob >= 0.5).numpy().astype(int).flatten()

# a) Create confusion matrix
cm_log = confusion_matrix(y_test, y_pred_log)  # SOLUTION

print("Confusion Matrix:")
print(cm_log)
print()

# b) Print classification report
print("Classification Report:")  # SOLUTION
print(classification_report(y_test, y_pred_log, target_names=['Class 0', 'Class 1']))

---

# Solutions Summary

## Exercise 1.1: Create Tensors
```python
tensor_a = torch.tensor([10, 20, 30, 40, 50])
tensor_b = torch.zeros(3, 4)
tensor_c = torch.rand(2, 5)
```

## Exercise 1.2: Tensor Operations
```python
result_add = x + y
result_mean = x.mean()
result_row_sum = x.sum(dim=1)
result_reshape = x.reshape(3, 2)
```

## Exercise 1.3: Autograd
```python
x = torch.tensor([2.0], requires_grad=True)
y = 3*x**2 - 4*x + 5
y.backward()
```

## Exercise 2.1: Linear Regression
```python
self.linear = nn.Linear(1, 1)
return self.linear(x)
my_criterion = nn.MSELoss()
my_optimizer = optim.SGD(my_model.parameters(), lr=0.01)
```

## Exercise 3.1: Logistic Regression
```python
self.linear = nn.Linear(input_dim, 1)
self.sigmoid = nn.Sigmoid()
return self.sigmoid(self.linear(x))
my_bce_loss = nn.BCELoss()
my_opt = optim.Adam(my_classifier.parameters(), lr=0.1)
```

## Exercise 4.1: SVM
```python
my_svm = SVC(kernel='rbf', C=10)
my_svm.fit(X_train_m, y_train_m)
```

## Exercise 5.1: Evaluation
```python
cm_log = confusion_matrix(y_test, y_pred_log)
print(classification_report(y_test, y_pred_log))
```